In [ ]:
import Pkg; Pkg.add(url="https://github.com/JuliaStats/StatsFuns.jl", rev="an/nopdf")
Pkg.add("SpecialFunctions")
using StatsFuns: betapdf,betalogpdf,betacdf, betaccdf,betalogcdf,betalogccdf,betainvcdf,betainvccdf,xlogy, log1p,logbeta
using SpecialFunctions: beta_inc

In [ ]:
function betalogpdf2(α::T, β::T, x::T) where T<:Real
    return if (x < 1 || x > 1)
        -Inf
    elseif x <= 0.5
        xlogy(α - 1, x) + (β - 1) * log1p(-x) - logbeta(α, β)
    else
        betalogpdf2(β, α, 1 - x)
    end
end

function betalogcdf2(α::Float64, β::Float64, x::Float64)
    if x < 0
        return -Inf
    elseif x > 1
        return 0
    else
        p, q = beta_inc(α, β, x)
        if p < eps(one(p))
            # see https://dlmf.nist.gov/8.17#E7
            return -log(α) + α*log(x) + log(_₂F₁(promote(α, 1 - β, α + 1, x)...)) - logbeta(α, β)
        elseif p <= 0.7
            return log(p)
        else
            return log1p(-q)
        end
    end
end
betalogcdf2(α::Real, β::Real, x::Real) = betalogcdf2(promote(float(α), β, x)...)
beta_inc2(α::Real, β::Real, x::Real) = beta_inc(promote(float(α), β, x)...)

In [ ]:
α=1.1;β=2.3;x=2.0
betalogpdf2(α,β,x)
betalogcdf2(α,β,x)


In [ ]:
using ForwardDiff, Zygote

In [ ]:
ForwardDiff.derivative(x->beta_inc2(α,β,x),0.3)

In [ ]:
import Pkg; Pkg.add.(["ModelingToolkit","GalacticOptim","Optim","ForwardDiff","Zygote","NonlinearSolve","RuntimeGeneratedFunctions"])

In [ ]:
using ModelingToolkit, GalacticOptim, Optim, ForwardDiff, Zygote, NonlinearSolve,RuntimeGeneratedFunctions,LinearAlgebra

In [ ]:
import ModelingToolkit: calculate_gradient, generate_gradient
function calculate_gradient(sys::NonlinearSystem)
    expand_derivatives.(gradient(equations(sys), states(sys)))
end

function generate_gradient(sys::NonlinearSystem, vs = states(sys), ps = parameters(sys); kwargs...)
    grad = calculate_gradient(sys)
    return build_function(grad, vs, ps;
                          conv = AbstractSysToExpr(sys),kwargs...)
end

In [ ]:
using SciMLBase,DiffEqBase,Symbolics, ModelingToolkit, GalacticOptim, Optim, ForwardDiff, Zygote, NonlinearSolve,RuntimeGeneratedFunctions,LinearAlgebra 
using UnPack: @unpack

module NonLinProbPrecompile
    using ModelingToolkit, LinearAlgebra

    function system(; kwargs...)
        # Define some variables
        A = Matrix(I,2,2); b = rand(2)
        @variables z[1:5]
        @parameters p[1:2]
        zcat = vcat(z...);
        pcat = vcat(p...);
    
        # Define a system of nonlinear equations
        ceq = vcat(0 .~ A*zcat[1:2].^4 - pcat, 0 .~ A*zcat[3:4] - pcat + vcat(z[5],z[2]))
        ns = NonlinearSystem(ceq,z,p)
        return generate_function(ns,z,p)
    end
    # Setting eval_expression=false and eval_module=[this module] will ensure
    # the RGFs are put into our own cache, initialised below.
    using RuntimeGeneratedFunctions
    RuntimeGeneratedFunctions.init(@__MODULE__)
    const f_noeval_good = system(; eval_expression=false, eval_module=@__MODULE__)
end

In [ ]:
f = eval(NonLinProbPrecompile.f_noeval_good[1])
z0=ones(5)
p0=[2.2,2.3]
@variables z[1:5]
@parameters p[1:2]
f_expr = f(z,p)
eq = 0 .~ f(z,p)
ns = NonlinearSystem(eq,z,p; name=:random_name, defaults=merge(Dict(vcat(z) .=> z0),Dict(vcat(p) .=> p0)) ) #NonlinearSystem(eq,z,p)
prob_ns = NonlinearProblem(ns,z0,p0; check_length=false,jac = true, sparse=true,checkbounds = false, linenumbers = false) #NonlinearProblem(ns,z0,p0; check_length=false)
# NonlinearFunction(ns,z0,p0)
@show states(ns);
@show parameters(ns);

In [ ]:
sys_num = generate_function(ns) 
sys_sym = generate_function(ns,z,p) 
jac_num = generate_jacobian(ns)
jac_sym = generate_jacobian(ns,z,p) 
jac_sym_sp = Symbolics.sparsejacobian(f(z,p),states(ns))
hess_sym = ModelingToolkit.hessian(f(z,p)[1],states(ns); simplify=true) # hessian of 1st equation in f
hess_num = build_function(hess_sym, states(ns); expression = false, target = Symbolics.JuliaTarget())

sn = @eval eval(sys_num[1])
sn_iip = @eval eval(sys_num[2])
ss = @eval eval(sys_sym[1])

jn = @eval eval(jac_num[1])
jn_iip = @eval eval(jac_num[2])
js = @eval eval(jac_sym[1])
js_sp = jac_sym_sp

hn = @eval eval(hess_num[1])
hn_iip = @eval eval(hess_num[2])
hs = @eval eval(hess_sym)

In [ ]:
@show sn(z0,p0)
@show ss(z,p)
@show jn(z0,p0)
@show js(z,p)
@show js_sp
@show hn(z0,p0)
@show hs

sn_z0 = zeros(length(z0)) 
jn_z0 = zeros(length(eq),length(z0)) 
hn_z0 = zeros(length(z0),length(z0)) 
@show sn_iip(sn_z0,z0,p0)
@show jn_iip(jn_z0,z0,p0)
@show hn_iip(hn_z0,z0,p0)

@show sn_z0
@show jn_z0
@show hn_z0

In [ ]:
ns_tear = tearing(ns;simplify=true)
ns_tear = tearing(ns;simplify=false)
states(ns_tear)
parameters(ns_tear)
s = structure(ns_tear)
@unpack fullvars, vartype, varassoc, inv_varassoc, varmask, algeqs, graph, solvable_graph, assign, inv_assign, scc, partitions = s
@show jac_spcy = ModelingToolkit.jacobian_sparsity(ns);
@show jac_spcy.colptr;
@show jac_spcy.rowval;

In [ ]:
zcat = vcat(z...)
loss =  sum(zcat).^3
op_sys = OptimizationSystem(loss,z,p,name=:sys1)

In [ ]:
@show equations(op_sys);
@show states(op_sys);
@show parameters(op_sys);

In [ ]:
op_grad_sym = calculate_gradient(op_sys)
op_hess_sym = calculate_hessian(op_sys)

op_loss_num = generate_function(op_sys)
op_grad_num = generate_gradient(op_sys)
op_hess_num = generate_hessian(op_sys)
op_hess_spcy = ModelingToolkit.hessian_sparsity(op_sys)

In [ ]:
display(op_grad_sym)
display(op_hess_sym)

op_ln = @eval eval(op_loss_num)

op_gn = @eval eval(op_grad_num[1])
op_gn_iip = @eval eval(op_grad_num[2])

op_hess = @eval eval(op_hess_num[1])
op_hess_iip = @eval eval(op_hess_num[2])
op_hess_sp = op_hess_spcy


In [ ]:
@show op_ln(z0,p0)
@show op_gn(z0,p0)
gn0 = zeros(length(z0))
op_gn_iip(gn0,z0,p0)
@show gn0;
@show op_hess(z0,p0)
hess0 = zeros(length(z0),length(z0))
op_hess_iip(hess0,z0,p0)
@show hess0
@show display(op_hess_sp )

In [ ]:
prob = OptimizationProblem(op_sys,z0,p0,grad=true,hess=true, lb=zeros(size(z0)), ub=ones(size(z0)) )
sol = solve(prob,Optim.Newton())

In [ ]:
# equality_constraints not implemented yet?
op_eq = [0 ~ z[1]]
op_sys = OptimizationSystem(loss,z,p,name=:sys1,equality_constraints=op_eq)

In [ ]:
# lb and ub
opt_f = OptimizationFunction(op_ln,GalacticOptim.AutoZygote())
opt_prob = OptimizationProblem(opt_f, z0, p0 , lb=zeros(size(z0)), ub=ones(size(z0)) )
solve(opt_prob,Optim.BFGS())           

In [ ]:
# lb and ub
opt_f = OptimizationFunction(op_ln,GalacticOptim.AutoForwardDiff())
opt_prob = OptimizationProblem(opt_f, z0, p0 , lb=zeros(size(z0)), ub=ones(size(z0)) )
solve(opt_prob,IPNewton())

In [ ]:
                          grad=nothing,
                          hess=nothing,
                          hv=nothing,
                          cons=nothing,
                          cons_j=nothing,
                          cons_h=nothing)

In [ ]:
import Pkg; 
Pkg.resolve();Pkg.gc();Pkg.precompile()


In [ ]:
1

In [ ]:
prob = OptimizationProblem(op_sys,z0,p0,grad=true,hess=true, lb=zeros(size(z0)), ub=ones(size(z0)),lcons=0,ucons=0.1 )
sol = solve(prob,Optim.Newton())
#OptimizationProblemExpr(op_sys, z0,p0, lb=zeros(size(z0)), ub=ones(size(z0)) ,grad=true,hess=true)

In [ ]:
calculate_gradient(op_sys)
calculate_hessian(op_sys)
generate_gradient(op_sys)
generate_hessian(op_sys)
ModelingToolkit.hessian_sparsity(op_sys)

In [ ]:
using ModelingToolkit, SparseArrays, Test, GalacticOptim, Optim


equations(combinedsys)
states(combinedsys)
parameters(combinedsys)

calculate_gradient(combinedsys)
calculate_hessian(combinedsys)
generate_function(combinedsys)
generate_gradient(combinedsys)
generate_hessian(combinedsys)
ModelingToolkit.hessian_sparsity(combinedsys)

u0 = [
    sys1.x=>1.0
    sys1.y=>2.0
    sys2.x=>3.0
    sys2.y=>4.0
    z=>5.0
]
p = [
    sys1.a => 6.0
    sys1.b => 7.0
    sys2.a => 8.0
    sys2.b => 9.0
    β => 10.0
]

prob = OptimizationProblem(combinedsys,u0,p,grad=true)
sol = solve(prob,NelderMead())
@test sol.minimum < -1e5

prob2 = remake(prob,u0=sol.minimizer)
sol = solve(prob,BFGS(initial_stepnorm=0.0001),allow_f_increases=true)
@test sol.minimum < -1e8
sol = solve(prob2,BFGS(initial_stepnorm=0.0001),allow_f_increases=true)
@test sol.minimum < -1e9

rosenbrock(x, p) =  (p[1] - x[1])^2 + p[2] * (x[2] - x[1]^2)^2
x0 = zeros(2)
_p  = [1.0, 100.0]

f = OptimizationFunction(rosenbrock,ModelingToolkit.AutoModelingToolkit(),x0,_p,grad=true,hess=true)
prob = OptimizationProblem(f,x0,_p)
sol = solve(prob,Optim.Newton())

In [1]:
import Pkg;Pkg.activate("optim")
Pkg.instantiate()
using ModelingToolkit, LinearAlgebra, Symbolics

  Activating environment at `~/SageMaker/networks_hub/SAN/code/network_simulations/optim/Project.toml`


In [503]:
N=5
M=N^2+4*N

for i=1:M
   @eval @parameters ($(Symbol("z$i"))) 
end
for i=1:N
   @eval @parameters ($(Symbol("x$i")))
   @eval @parameters ($(Symbol("p_bar$i")))
end
@parameters γ
zvec = [ @eval ($(Symbol("z$i"))) for i=1:M]
p_barvec = [ @eval ($(Symbol("p_bar$i"))) for i=1:N]
xvec = [ @eval ($(Symbol("x$i"))) for i=1:N]
Aᵀ = Array(transpose(reshape(zvec[1:N^2],N,N)))
c = zvec[N^2+N+1:N^2+2N]
#z = vcat(A...,b,c,α,β)


5-element Vector{Num}:
 z31
 z32
 z33
 z34
 z35

In [504]:
for i=1:N
   @eval @parameters $(Symbol("p$i"))($(zvec...))
end
pvec = [ @eval $(Symbol("p$i")) for i=1:N]

5-element Vector{Num}:
 p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45)
 p2(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45)
 p3(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45)
 p4(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45)
 p5(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z2

In [505]:
#eq = Aᵀ*pvec - pvec
#min.( (1+γ)*(Aᵀ*pvec + (1 .- x).*c) - γ*p_bar, p_bar)

eq = min.(((1+γ)*(Aᵀ*pvec + (1 .- xvec).*c) - γ*p_barvec),p_barvec) - pvec


5-element Vector{Num}:
      min((1 + γ)*(z1*p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45) + z2*p2(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45) + z3*p3(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45) + z31*(1 - x1) + z4*p4(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30, z31, z32, z33, z34, z35, z36, z37, z38, z39, z40, z41, z42, z43, z44, z45) + z5*p5(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, 

In [288]:
deq = Symbolics.jacobian(eq,zvec);

In [289]:
deq[1]

IfElse.ifelse(((1 + γ)*(z1*p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21) + z13*(1 - x1) + z2*p2(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21) + z3*p3(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) - (p_bar1*γ)) < p_bar1, 1, 0)*(1 + γ)*(z1*Differential(z1)(p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) + z2*Differential(z1)(p2(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) + z3*Differential(z1)(p3(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) + p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) - Differential(z1)(p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21))

In [290]:
#_deq = expand_derivatives.(deq);

In [291]:
g_eq=Symbolics.jacobian(eq,zvec);

In [292]:
@show size(g_eq)
g_eq = reshape(g_eq,N*M);

size(g_eq) = (3, 21)


In [293]:
D = Differential(z1)
@show D(pvec[1])
@show g_eq[1];

D(pvec[1]) = Differential(z1)(p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21))
g_eq[1] = IfElse.ifelse(((1 + γ)*(z1*p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21) + z13*(1 - x1) + z2*p2(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21) + z3*p3(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) - (p_bar1*γ)) < p_bar1, 1, 0)*(1 + γ)*(z1*Differential(z1)(p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) + z2*Differential(z1)(p2(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) + z3*Differential(z1)(p3(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) + p1(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21)) - Dif

In [506]:
for i=1:M
    @eval $(Symbol("D$i")) = Differential($(Symbol("z$i")))
    for j=1:N
        @eval @parameters $(Symbol("d$i$j"))
    end
end
dvec = [ @eval ($(Symbol("d$i$j"))) for i=1:M, j=1:N];

In [507]:
subs = Dict{Num,Num}()
for i=1:M
    for j=1:N
        temp = @eval Dict([Differential("z$i")(pvec[$j])=> eval(Symbol("d$i$j"))])
        merge!(subs,temp)
    end
end



In [508]:
dict_Dz = Dict{Num,Num}()
for i=1:M
    for j=1:N
        temp = @eval Dict([Differential($(Symbol("z$i")))(pvec[$j]) => $(Symbol("d$i$j"))])
        merge!(dict_Dz,temp)
    end
end

for j=1:N
    @eval @variables $(Symbol("pclear$j"))
end
pclearvec = [ @eval ($(Symbol("pclear$i"))) for i=1:N]

dict_p = Dict{Num,Num}()
for j=1:N
    temp = @eval Dict([pvec[$j] => $(Symbol("pclear$j")) ])
    merge!(dict_p,temp)
end


In [297]:
lin_sys = [0 ~ Symbolics.substitute(g_eq[i],merge(dict_Dz,dict_p)) for i=1:M*N]
dvec = [@eval $(Symbol("d$i$j")) for i=1:M for j=1:N];

In [300]:
@show size(lin_sys)
lin_sys[1]
Symbolics.solve_for(lin_sys[1],d11)

size(lin_sys) = (63,)


-((z1*IfElse.ifelse(((1 + γ)*(pclear1*z1 + pclear2*z2 + pclear3*z3 + z13*(1 - x1)) - (p_bar1*γ)) < p_bar1, 1, 0)*(1 + γ) - 1)^-1)*IfElse.ifelse(((1 + γ)*(pclear1*z1 + pclear2*z2 + pclear3*z3 + z13*(1 - x1)) - (p_bar1*γ)) < p_bar1, 1, 0)*(1 + γ)*(pclear1 + d12*z2 + d13*z3)

In [ ]:
sol = Symbolics.solve_for(lin_sys,dvec;simplify=true);

In [274]:
sol[1];

In [302]:
lin_sys[1]

0 ~ IfElse.ifelse(((1 + γ)*(pclear1*z1 + pclear2*z2 + pclear3*z3 + z13*(1 - x1)) - (p_bar1*γ)) < p_bar1, 1, 0)*(1 + γ)*(pclear1 + d11*z1 + d12*z2 + d13*z3) - d11

In [543]:
tril!(trues(N,N), -1)

5×5 BitMatrix:
 0  0  0  0  0
 1  0  0  0  0
 1  1  0  0  0
 1  1  1  0  0
 1  1  1  1  0

In [627]:
li = LinearIndices((N,N))
liᵀ= transpose(li)
id = li[tril!(trues(N,N), -1)]
idᵀ = liᵀ[tril!(trues(N,N), -1)]

lt = (LowerTriangular(Aᵀ)-Diagonal(Aᵀ))[id] .=> 0
ut = (UpperTriangular(Aᵀ)-Diagonal(Aᵀ))[idᵀ] .=> 0

nz = N*(N-1)÷2
pick = rand(Bool,nz)
lut = vcat(lt[pick],ut[.!(pick)])

10-element Vector{Pair{Num, Int64}}:
  z6 => 0
 z16 => 0
 z22 => 0
 z18 => 0
 z23 => 0
 z24 => 0
  z3 => 0
  z5 => 0
  z8 => 0
  z9 => 0

In [628]:
dict_γ = Dict([γ=>0])
dict_A = Dict(diag(Aᵀ).=> 0)
dict_zero = Dict(lut)
fp = [Symbolics.substitute(eq[i],merge(dict_Dz,dict_p,dict_γ,dict_A,dict_zero)) for i=1:N]
[Symbolics.substitute(Aᵀ[i,j],merge(dict_A,dict_zero)) for i=1:N, j=1:N]

5×5 Matrix{Num}:
   0   z2  0   z4    0
   0    0  0    0  z10
 z11  z12  0  z14  z15
   0  z17  0    0  z20
 z21    0  0    0    0

In [682]:
ns_eq = 0 .~ fp
ns = NonlinearSystem(ns_eq, pclearvec,vcat(zvec...,xvec...,p_barvec...))

Model ##NonlinearSystem#124275 with 5 equations
States (5):
  pclear1
  pclear2
  pclear3
  pclear4
  pclear5
Parameters (55):
  z1
  z2
  z3
  z4
  z5
  z6
⋮

In [683]:
ns_old = ns
ns_new = structural_simplify(ns)
count=1
while ns_old != ns_new
    ns_old=ns_new
    ns_new=structural_simplify(ns_new)
    count += 1
    @show count
end   
ns_new    

count = 2
count = 3


Model ##NonlinearSystem#124275 with 1 equations
States (1):
  pclear5
Parameters (55):
  z1
  z2
  z3
  z4
  z5
  z6
⋮
Incidence matrix:
 ×

In [684]:
states(ns_new)

1-element Vector{Any}:
 pclear5

In [685]:
using NonlinearSolve
guess = states(ns_new) .=> 1.0
params = parameters(ns_new) .=> rand(length(parameters(ns_new)))


55-element Vector{Pair{Sym{Real, Base.ImmutableDict{DataType, Any}}, Float64}}:
     z1 => 0.07479366447488212
     z2 => 0.2607268729193257
     z3 => 0.4927678084197209
     z4 => 0.13883751223836427
     z5 => 0.8076584483256972
     z6 => 0.9680514399112186
     z7 => 0.006172191841003594
     z8 => 0.901647265277506
     z9 => 0.08345093711943918
    z10 => 0.08664643838336761
    z11 => 0.15588875937739455
    z12 => 0.9345000088580333
    z13 => 0.6723650802127072
        ⋮
    z44 => 0.9014792285817244
    z45 => 0.009239807298989211
     x1 => 0.9432577367771464
     x2 => 0.6109814444982788
     x3 => 0.9343221013025402
     x4 => 0.1418887485567899
     x5 => 0.6664650028913848
 p_bar1 => 0.9208219549388541
 p_bar2 => 0.7005649889023702
 p_bar3 => 0.6847395809503594
 p_bar4 => 0.7486066720659037
 p_bar5 => 0.8085322640698109

In [686]:
using LinearAlgebra
BLAS.set_num_threads(32)

prob = NonlinearProblem(ns_new,guess,params)
sol = solve(prob,NewtonRaphson())

u: 1-element Vector{Float64}:
 0.06595074068218666

In [707]:
ns_new.states

vcat(setdiff(pclearvec,ns_new.states)...)
ns_new.states .=> sol.u
Dict(ns_new.states .=> sol.u)


4-element Vector{Equation}:
 pclear2 ~ min(0.06595074068218666z10 + z32*(1 - x2), p_bar2)
 pclear4 ~ min(z17*min(0.06595074068218666z10 + z32*(1 - x2), p_bar2) + 0.06595074068218666z20 + z34*(1 - x4), p_bar4)
 pclear1 ~ min(pclear2*z2 + pclear4*z4 + z31*(1 - x1), p_bar1)
 pclear3 ~ min(pclear1*z11 + pclear2*z12 + pclear4*z14 + 0.06595074068218666z15 + z33*(1 - x3), p_bar3)

In [712]:
ns_num = [Symbolics.substitute(ns_new.observed[i],Dict(ns_new.states .=> sol.u)) for i=1:length(ns_new.observed)]
ns_obs_old = NonlinearSystem(ns_num, setdiff(pclearvec,ns_new.states),vcat(zvec...,xvec...,p_barvec...))
ns_obs_new = structural_simplify(ns_obs_old)

Model ##NonlinearSystem#124283 with 0 equations
States (0):
Parameters (55):
  z1
  z2
  z3
  z4
  z5
  z6
⋮
Incidence matrix:


In [728]:
alias_elimination(ns_obs_old)


Model ##NonlinearSystem#124283 with 4 equations
States (4):
  pclear2
  pclear4
  pclear1
  pclear3
Parameters (55):
  z1
  z2
  z3
  z4
  z5
  z6
⋮

In [715]:
ns_obs_old

Model ##NonlinearSystem#124283 with 4 equations
States (4):
  pclear1
  pclear2
  pclear3
  pclear4
Parameters (55):
  z1
  z2
  z3
  z4
  z5
  z6
⋮

In [ ]:
dump(sim_sys)

In [433]:
asdigraph(sim_sys.structure.solvable_graph)

LoadError: MethodError: no method matching asdigraph(::BipartiteGraph{Int64, Vector{Vector{Int64}}, Int64, Nothing})
[0mClosest candidates are:
[0m  asdigraph(::BipartiteGraph, [91m::ModelingToolkit.AbstractSystem[39m; variables, equationsfirst) at /home/ec2-user/.julia/packages/ModelingToolkit/45oYI/src/systems/dependency_graphs.jl:188

In [640]:
tear_sys =  tearing(ns)
tearing

LoadError: syntax: space required before "?" operator

In [405]:
sys = initialize_system_structure(ns)
StructuralTransformations.find_solvables!(sys)
sss = structure(sys)
@unpack graph, solvable_graph, fullvars = sss


Incidence matrix:
 ×  ×  ×
 ⋅  ×  ×
 ⋅  ⋅  ×

In [678]:
dump(ns_new)

NonlinearSystem
  eqs: Array{Equation}((1,))
    1: Equation
      lhs: Int64 0
      rhs: SymbolicUtils.Add{Real, Int64, Dict{Any, Number}, Nothing}
        coeff: Int64 0
        dict: Dict{Any, Number}
          slots: Array{UInt8}((16,)) UInt8[0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]
          keys: Array{Any}((16,))
            1: #undef
            2: Term{Real, Nothing}
              f: min (function of type typeof(min))
              arguments: Array{SymbolicUtils.Symbolic{Real}}((2,))
                1: SymbolicUtils.Add{Real, Int64, Dict{Any, Number}, Nothing}
                2: Sym{Real, Base.ImmutableDict{DataType, Any}}
              metadata: Nothing nothing
              hash: Base.RefValue{UInt64}
                x: UInt64 0x23f41dbf84dfc3e1
            3: #undef
            4: #undef
            5: #undef
            ...
            12: #undef
            13: #undef
            14: #undef
            15: #undef
   

In [381]:
using ModelingToolkit.BipartiteGraphs
big_rc = initialize_system_structure(big_rc)
inc_org = BipartiteGraphs.incidence_matrix(structure(big_rc).graph)
blt_org = StructuralTransformations.sorted_incidence_matrix(big_rc, only_algeqs=true, only_algvars=true)
blt_reduced = StructuralTransformations.sorted_incidence_matrix(sys, only_algeqs=true, only_algvars=true)

1-element Vector{Equation}:
 0 ~ min(z15*(1 - x3) + z7*min(z13*(1 - x1) + z2*min(pclear3*z6 + z14*(1 - x2), p_bar2), p_bar1), p_bar3) - pclear3

In [285]:
prob = NonlinearProblem(sim_sys,[1.0,2.0,3.0],ps)
sol = solve(prob,NewtonRaphson())

In [285]:
Dk = Differential(k1)
eq = min.([k1^2,k2],[ub1,ub2])

2-element Vector{Num}:
 min(k1^2, ub1)
   min(k2, ub2)

In [286]:
expand_derivatives.(Dk.(eq))

2-element Vector{Num}:
 2k1*IfElse.ifelse((k1^2) < ub1, 1, 0)
     0